In [6]:
# resdata - Jupyter Notebook analysis pipeline test
The testing cells.
1. Fetch single ring by id and visualise the data
2. Fetch relevant data from full experiment and visualise as histogram
3. Fetch relevant data from full experiment and visualise

SyntaxError: invalid syntax (1157603407.py, line 2)

In [ ]:
## Sing in and Authentication
username = 'test_user_main'
password = 'some_password'

In [4]:
import interface as ui



In [ ]:
# Initialisation cell
'''
Establish connection to the machine and verify the connection using the interface
'''


# update the variables file 